In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:11pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

<b><font size="6" color="red">Ch.9-1 Vector Embedding Model</font><b>

# 문장 → 임베딩 벡터(1차원 숫자 배열)
- openAI API (.env - OPENAI_API_KEY) - text-embedding-3-large
- upstage API (https://console.upstage.ai/docs/getting-started) (.env - UPSTAGE_API_KEY) 
- 소득세법 chunking

# 1. 환경변수 load

In [4]:
from dotenv import load_dotenv
import os
load_dotenv(
#    dotenv_path='.env' #동일 경로이므로 생략 가능
)
openai_key = os.getenv('OPENAI_API_KEY')
upstage_key = os.getenv('UPSTAGE_API_KEY')
print(openai_key[:2])
print(upstage_key[:2])

sk
up


# 2. 유사도 계산하는 방법
1. 유클리드 거리 : 두 벡터 간 거리가 가까운지
2. ✔ 코사인 유사도 : 두 벡터 간 방향이 유사한지
3. dot product : 두 벡터 간 곱을 사용하여 거리와 방향을 모두 고려

In [6]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) 
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)

# 3. openAI API의 Embedding Model

In [7]:
from openai import OpenAI
openai_client = OpenAI(
    # api_key = openai_key
)

In [8]:
# text-embedding-3-large (과금)
response = openai_client.embeddings.create(
    input = "The king is the prince's father.",
    model = "text-embedding-3-large"
)

In [10]:
response.usage

Usage(prompt_tokens=8, total_tokens=8)

In [12]:
king_vector = np.array(response.data[0].embedding)
print(king_vector.shape)
print(king_vector)

(3072,)
[ 0.02944428  0.02673196 -0.01142302 ...  0.00223636  0.01163166
 -0.00231297]


In [13]:
queen_response = openai_client.embeddings.create(
    input = "The queen is the prince's mother",
    model = "text-embedding-3-large"
)

In [14]:
queen_response.usage

Usage(prompt_tokens=7, total_tokens=7)

In [15]:
queen_vector = np.array(queen_response.data[0].embedding)
print(queen_vector.shape)
print(queen_vector)

(3072,)
[ 0.01908419  0.00117847 -0.00398764 ...  0.00576628  0.00822409
  0.01082648]


In [16]:
# king_vector와 queen_response 유사도
k_q_similarity = cosine_similarity(king_vector, queen_vector)
print("king 문장과 queen 문장의 유사도:", k_q_similarity)

king 문장과 queen 문장의 유사도: 0.6519634954980786


In [17]:
clown_response = openai_client.embeddings.create(
    input = "The clown sings",
    model = "text-embedding-3-large"
)

In [18]:
clown_vector = np.array(clown_response.data[0].embedding)
print(clown_vector.shape)
print(clown_vector)

(3072,)
[ 0.02284988 -0.01164881 -0.00676233 ... -0.00125153  0.00570861
  0.00455454]


In [19]:
k_c_similarity = cosine_similarity(king_vector, clown_vector)
print("king 문장과 clown 문장 유사도:", k_c_similarity)

king 문장과 clown 문장 유사도: 0.11948664675200178


In [ ]:
# 한국어 문장을 벡터로 바꿔도 유사도가...?

In [20]:
kor_response = openai_client.embeddings.create(
    input = "왕은 왕자의 아버지다",
    model = "text-embedding-3-large"
)

In [21]:
kor_k_vector = np.array(kor_response.data[0].embedding)
print(kor_k_vector.shape)
print(kor_k_vector)

(3072,)
[0.01460524 0.0081415  0.00299669 ... 0.00721534 0.01149217 0.00849063]


In [22]:
kor_q_response = openai_client.embeddings.create(
    input = "여왕은 왕자의 어머니다",
    model = "text-embedding-3-large"
)

In [23]:
kor_q_vector = np.array(kor_q_response.data[0].embedding)
print(kor_q_vector.shape)
print(kor_q_vector)

(3072,)
[-0.00633463  0.01295298  0.00534164 ... -0.00638844 -0.00050108
  0.0337521 ]


In [24]:
print("왕 문장과 여왕 문장의 유사도:", cosine_similarity(kor_k_vector, kor_q_vector))

왕 문장과 여왕 문장의 유사도: 0.5744286874279486


In [25]:
kor_c_response = openai_client.embeddings.create(
    input = "광대가 노래한다",
    model = "text-embedding-3-large"
)

In [26]:
kor_c_vector = np.array(kor_c_response.data[0].embedding)
print(kor_c_vector.shape)
print(kor_c_vector)

(3072,)
[ 0.01472999 -0.03652103 -0.00336393 ... -0.00793783  0.00575171
  0.01328037]


In [28]:
kor_k_c_similarity = cosine_similarity(kor_k_vector, kor_c_vector)
print("왕 문장과 광대 문장 유사도:", kor_k_c_similarity)

왕 문장과 광대 문장 유사도: 0.2380159535776867


In [29]:
king_vector

array([ 0.02944428,  0.02673196, -0.01142302, ...,  0.00223636,
        0.01163166, -0.00231297], shape=(3072,))

In [30]:
kor_k_vector

array([0.01460524, 0.0081415 , 0.00299669, ..., 0.00721534, 0.01149217,
       0.00849063], shape=(3072,))

In [31]:
# king_vector와 kor_k_vector의 유사도
print("king 문장과 왕 문장의 유사도:", cosine_similarity(king_vector, kor_k_vector))

king 문장과 왕 문장의 유사도: 0.6447090950163226


# 4. upstage의 Embedding Model
- 한국어에서는 openai보다 embedding 성능이 좋다.

In [32]:
# from openai import OpenAI # openai==1.52.2 
upstage_client = OpenAI(
    api_key=upstage_key, #생략하면 openai_key 들어감
    base_url="https://api.upstage.ai/v1"
)

In [34]:
response = upstage_client.embeddings.create(
    input="The king is prince's father",
    model="embedding-query"
)

In [37]:
up_king_vector = np.array(response.data[0].embedding)
print(up_king_vector.shape)
print(up_king_vector)

(4096,)
[ 8.99434090e-05 -3.04260254e-02 -2.86674500e-03 ... -7.19451904e-03
  1.04446411e-02  3.69873047e-02]


In [38]:
response = upstage_client.embeddings.create(
    input="The queen is prince's mother",
    model="embedding-query"
)

In [39]:
up_queen_vector = np.array(response.data[0].embedding)
print(up_queen_vector.shape)
print(up_queen_vector)

(4096,)
[-0.00610733 -0.01773071  0.00730896 ...  0.00306511 -0.00260544
  0.04815674]


In [40]:
print("king 문장과 queen 문장의 유사도:", cosine_similarity(up_king_vector, up_queen_vector))

king 문장과 queen 문장의 유사도: 0.6695590665947906


In [41]:
response = upstage_client.embeddings.create(
    input="왕은 왕자의 아버지다",
    model="embedding-query"
)
up_kor_k_vec = np.array(response.data[0].embedding)

In [42]:
print(up_kor_k_vec.shape)
print(up_kor_k_vec)

(4096,)
[-0.00049489 -0.02248897 -0.01110534 ... -0.00108751  0.01807599
  0.01477619]


In [43]:
response = upstage_client.embeddings.create(
    input="여왕은 왕자의 어머니다",
    model="embedding-query"
)

In [44]:
up_kor_q_vec = np.array(response.data[0].embedding)
print(up_kor_q_vec.shape)
print(up_kor_q_vec)

(4096,)
[-0.01261867 -0.00149708 -0.00043387 ... -0.0008137  -0.00722157
  0.02597476]


In [45]:
print("왕 문장과 여왕 문장의 유사도:", cosine_similarity(up_kor_k_vec, up_kor_q_vec))

왕 문장과 여왕 문장의 유사도: 0.6502829514419388


In [46]:
print("king 문장과 왕 문장의 유사도:", cosine_similarity(up_king_vector, up_kor_k_vec))

king 문장과 왕 문장의 유사도: 0.8494802298580414


In [47]:
print("queen 문장과 여왕 문장의 유사도:", cosine_similarity(up_queen_vector, up_kor_q_vec))

queen 문장과 여왕 문장의 유사도: 0.802447131719098
